In [11]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic
import json

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"
num_tokens = 1000

In [5]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": num_tokens,
        "messages": messages,
        "temperature": temperature,
    }

    if system:
        params["system"] = system
    
    if stop_sequences:
        params["stop_sequences"] = stop_sequences

    # message = client.messages.create(**params)
    with client.messages.stream(**params) as stream:
        for text in stream.text_stream:
            pass
    message = stream.get_final_message()
    return message.content[0].text

In [ ]:
# Function to generate a new dataset
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [4]:
# Generate the dataset and write it to 'dataset.json'
dataset = generate_dataset()
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3667, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_2429713/3627977199.py", line 2, in <module>
    dataset = generate_dataset()
              ^^^^^^^^^^^^^^^^
NameError: name 'generate_dataset' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 2176, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "/home/jx-creator/mambaforge/envs/ctorch/lib/python3.13/site-packages/IPython/core/ultratb.py", line 1182, in structured_traceback
    return FormattedTB.structured_traceback(
          

In [ ]:
# Function to grade a test case + output using a model
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>



Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [7]:
# Passes a test case into Claude
def run_prompt(test_case):
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""

    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [8]:
# Function to execute a single test case and grade the output
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning,
    }

In [9]:
from statistics import mean


def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [10]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 7.666666666666667


In [18]:
print(json.dumps(results, indent=2))

[
  {
    "output": "Here's a Python function that converts AWS EC2 instance state codes to their human-readable descriptions:\n\n```python\ndef convert_ec2_instance_state(state_code):\n    \"\"\"\n    Convert AWS EC2 instance state code to human-readable description.\n    \n    Args:\n        state_code (int): The EC2 instance state code\n    \n    Returns:\n        str: Human-readable description of the instance state\n    \"\"\"\n    # Dictionary mapping state codes to their descriptions\n    state_map = {\n        0: 'pending',\n        16: 'running',\n        32: 'shutting-down',\n        48: 'terminated',\n        64: 'stopping',\n        80: 'stopped'\n    }\n    \n    # Return the description, or 'Unknown' if state code is not in the map\n    return state_map.get(state_code, 'Unknown')\n\n# Example usage\ndef main():\n    # Test the function with various state codes\n    test_codes = [0, 16, 32, 48, 64, 80, 99]\n    \n    for code in test_codes:\n        state = convert_ec2_ins

In [19]:
with open("results.json", "w") as f:
   json.dump(results, f, indent=2)